In [1]:
from linkml_runtime import SchemaView
import csv

In [2]:
schema_url = "https://raw.githubusercontent.com/GenomicsStandardsConsortium/mixs/refs/heads/main/src/mixs/schema/mixs.yaml"

In [3]:
class_slot_rank_tsv_file = "mixs_slot_rank_template.tsv"
class_ancestry_tsv = "mixs_class_ancestry.tsv"

In [4]:
schema_view = SchemaView(schema_url)

In [5]:
schema_classnames = list(schema_view.all_classes().keys())

In [6]:
schema_classnames.sort()

In [7]:
class_slot_rank_rows = []
class_ancestry_rows = []
checklist_tally = 0
extension_tally = 0
combination_tally = 0

In [8]:
# ~ 40 seconds
for cn in schema_classnames:
    ic = schema_view.induced_class(cn)
    ic_parent = ic.is_a
    if ic_parent == "Checklist":
        checklist_tally += 1
    elif ic_parent == "Extension":
        extension_tally += 1
    ic_mixins = ic.mixins
    if len(ic_mixins) > 0:
        combination_tally += 1
    class_ancestry_row = {
        "class": cn,
        "parent": ic_parent,
        "mixins": "|".join(ic_mixins)
    }
    class_ancestry_rows.append(class_ancestry_row)
    if ic_parent in ["Checklist", "Extension"] or len(ic_mixins) > 0:
        ic_slot_names = list(ic.attributes.keys())
        ic_slot_names.sort()
        for sn in ic_slot_names:
            class_slot_rank_row = {
                "class": cn,
                "slot": sn,
                "rank": ""
            }
            class_slot_rank_rows.append(class_slot_rank_row)


In [9]:
# use csv dictwriter to save the rows list of dicts to as TSV file whose name should come from tsv_file
fieldnames = ["class", "slot", "rank"]
with open(class_slot_rank_tsv_file, "w", newline="", encoding="utf-8") as outfile:
    writer = csv.DictWriter(outfile, fieldnames=fieldnames, delimiter="\t")
    writer.writeheader()
    writer.writerows(class_slot_rank_rows)


In [10]:
print(f"{checklist_tally = }")
print(f"{extension_tally = }")
print(f"{combination_tally = }")
print(f"{len(class_slot_rank_rows) = }")

checklist_tally = 11
extension_tally = 23
combination_tally = 253
len(class_slot_rank_rows) = 33457


2025-02-19: 33457 slots used by 23 Extension, 11 Checklist or 253 "combination" classes (which have mixins)

In [11]:
# use csv dictwriter to save the rows list of dicts to as TSV file whose name should come from tsv_file
fieldnames = ["class", "parent", "mixins"]
with open(class_ancestry_tsv, "w", newline="", encoding="utf-8") as outfile:
    writer = csv.DictWriter(outfile, fieldnames=fieldnames, delimiter="\t")
    writer.writeheader()
    writer.writerows(class_ancestry_rows)